# ***CIFAR-10 Train AI Model***

In [ ]:
# -*- coding: utf-8 -*-
"""
VGG11 from scratch pour CIFAR-10 (32x32), simple — sans régularisation, sans callbacks
"""

import time
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

In [ ]:
# Affiche si un GPU est détecté (TF choisit automatiquement)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("✅ GPU détecté :", gpus)
else:
    print("⚠️ Pas de GPU détecté — entraînement sur CPU.")

✅ GPU détecté : [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
# Timer minimal
class timer:
    def __init__(self):
        self.start = None
        self.stop = None
    def tic(self):
        self.start = time.time()
    def toc(self):
        self.stop = time.time()
    def res(self):
        return None if self.start is None or self.stop is None else self.stop - self.start

In [ ]:
# VGG11 simple (pas de BatchNorm, pas de dropout)
def build_vgg11(input_shape):
    model = models.Sequential(name="VGG11_CIFAR10_simple")
    model.add(layers.Input(shape=input_shape))


    model.add(layers.Conv2D(32, (3,3), padding='same', use_bias=True))
    model.add(layers.Activation('relu'))
    model.add(layers.BatchNormalization())

    model.add(layers.Conv2D(32, (3,3), padding='same', use_bias=True))
    model.add(layers.Activation('relu'))
    model.add(layers.SpatialDropout2D(0.25))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2,2)))

    model.add(layers.Conv2D(64, (3,3), padding='same', use_bias=True))
    model.add(layers.Activation('relu'))
    model.add(layers.BatchNormalization())

    model.add(layers.Conv2D(64, (3,3), padding='same', use_bias=True))
    model.add(layers.Activation('relu'))
    model.add(layers.SpatialDropout2D(0.25))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2,2)))

    model.add(layers.Conv2D(128, (3,3), padding='same', use_bias=True))
    model.add(layers.Activation('relu'))
    model.add(layers.SpatialDropout2D(0.25))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2,2)))

    model.add(layers.Conv2D(128, (3,3), padding='same', use_bias=True))
    model.add(layers.Activation('relu'))
    model.add(layers.SpatialDropout2D(0.25))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2,2)))

    model.add(layers.Flatten())
    model.add(layers.Dense(1024, activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(10, activation='softmax'))

    return model

In [ ]:
# VGG11 simple (pas de BatchNorm, pas de dropout)
def build_vgg11(input_shape):
    model = models.Sequential(name="VGG11_CIFAR10_simple")
    model.add(layers.Input(shape=input_shape))


    model.add(layers.Conv2D(16, (3,3), padding='same', use_bias=True)) # 32 de base
    model.add(layers.Activation('relu'))
    model.add(layers.BatchNormalization())

    model.add(layers.Conv2D(32, (3,3), padding='same', use_bias=True))
    model.add(layers.Activation('relu'))
    model.add(layers.SpatialDropout2D(0.25))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2,2)))

    model.add(layers.Conv2D(32, (3,3), padding='same', use_bias=True)) # 64 de base
    model.add(layers.Activation('relu'))
    model.add(layers.BatchNormalization())

    model.add(layers.Conv2D(64, (3,3), padding='same', use_bias=True))
    model.add(layers.Activation('relu'))
    model.add(layers.SpatialDropout2D(0.25))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2,2)))

    model.add(layers.Conv2D(64, (3,3), padding='same', use_bias=True)) #128 de base
    model.add(layers.Activation('relu'))
    model.add(layers.SpatialDropout2D(0.25))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2,2)))

    model.add(layers.Conv2D(128, (3,3), padding='same', use_bias=True))
    model.add(layers.Activation('relu'))
    model.add(layers.SpatialDropout2D(0.25))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2,2)))

    model.add(layers.GlobalAveragePooling2D()) #Flatten de base
    model.add(layers.Dense(256, activation='relu')) #1024 de base + une autre couche Dense avec 512
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(10, activation='softmax'))

    return model

In [ ]:
# Chargement et préparation des données CIFAR-10
class dataset:
    def __init__(self, nb_epochs=20, batch_size=64):
        (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

        x_train = x_train.astype('float32') / 255.0
        x_test  = x_test.astype('float32') / 255.0

        self.x_train = x_train
        self.y_train = y_train

        self.x_test = x_test
        self.y_test = y_test

        self.input_shape = self.x_train.shape[1:]
        self.nb_epochs = nb_epochs
        self.batch_size = batch_size

        # one-hot
        self.y_train = tf.keras.utils.to_categorical(self.y_train, 10)
        self.y_test = tf.keras.utils.to_categorical(self.y_test, 10)

        print("Train:", self.x_train.shape, self.y_train.shape)
        print("Test: ", self.x_test.shape, self.y_test.shape)
        print("Epochs:", self.nb_epochs, " Batch size:", self.batch_size)

In [ ]:
# Entraînement simple
def train_model(data):
    print("➡️ Construction du modèle...")
    model = build_vgg11(data.input_shape)

    # Optimiseur simple
    opt = tf.keras.optimizers.Adam(learning_rate=0.001)

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()

    t = timer()
    t.tic()
    history = model.fit(
        data.x_train, data.y_train,
        validation_data=(data.x_test, data.y_test),
        epochs=data.nb_epochs,
        batch_size=data.batch_size,
        shuffle=True,
        verbose=2
    )
    t.toc()
    print(f"✅ Entraînement terminé en {t.res():.1f} s")
    return model, history

In [ ]:
# Évaluation
def test_model(data, model):
    loss, acc = model.evaluate(data.x_test, data.y_test, verbose=2)
    print(f"Test loss: {loss:.4f}  Test accuracy: {acc:.4f}")

# (Facultatif) affichage des courbes
def plot_history(history):
    plt.figure(figsize=(10,4))
    plt.subplot(1,2,1)
    plt.plot(history.history['loss'], label='train loss'); plt.plot(history.history['val_loss'], label='val loss')
    plt.legend(); plt.title('Loss')
    plt.subplot(1,2,2)
    plt.plot(history.history['accuracy'], label='train acc'); plt.plot(history.history['val_accuracy'], label='val acc')
    plt.legend(); plt.title('Accuracy')
    plt.show()

In [ ]:
# MAIN
if __name__ == '__main__':
    # Hyperparamètres simples
    NB_EPOCHS = 50
    BATCH_SIZE = 64  # si VRAM limitée -> réduire à 32 ou 16

    data = dataset(nb_epochs=NB_EPOCHS, batch_size=BATCH_SIZE)
    model, history = train_model(data)
    test_model(data, model)

Train: (50000, 32, 32, 3) (50000, 10)
Test:  (10000, 32, 32, 3) (10000, 10)
Epochs: 50  Batch size: 64
➡️ Construction du modèle...


Model: "VGG11_CIFAR10_simple"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 32, 32, 16)     │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_6 (Activation)       │ (None, 32, 32, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 32, 32, 16)     │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 32, 32, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_7 (Activation)       │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout2d_4             │ (None, 32, 32, 32)     │             0 │
│ (SpatialDropout2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 32, 32, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 16, 16, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_8 (Activation)       │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 16, 16, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 16, 16, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_9 (Activation)       │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout2d_5             │ (None, 16, 16, 64)     │             0 │
│ (SpatialDropout2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 16, 16, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 8, 8, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_10 (Activation)      │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout2d_6             │ (None, 8, 8, 64)       │             0 │
│ (SpatialDropout2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ (None, 8, 8, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 4, 4, 64)       │             

 Total params: 180,554 (705.29 KB)

 Trainable params: 179,882 (702.66 KB)

 Non-trainable params: 672 (2.62 KB)

Epoch 1/50
782/782 - 25s - 32ms/step - accuracy: 0.3149 - loss: 1.8417 - val_accuracy: 0.4583 - val_loss: 1.4550
Epoch 2/50
782/782 - 25s - 33ms/step - accuracy: 0.4496 - loss: 1.4883 - val_accuracy: 0.5532 - val_loss: 1.2365
Epoch 3/50
782/782 - 10s - 13ms/step - accuracy: 0.5250 - loss: 1.3044 - val_accuracy: 0.5815 - val_loss: 1.1443
Epoch 4/50
782/782 - 5s - 7ms/step - accuracy: 0.5747 - loss: 1.1852 - val_accuracy: 0.6329 - val_loss: 1.0160
Epoch 5/50
782/782 - 10s - 13ms/step - accuracy: 0.6062 - loss: 1.1005 - val_accuracy: 0.6869 - val_loss: 0.8796
Epoch 6/50
782/782 - 5s - 7ms/step - accuracy: 0.6321 - loss: 1.0408 - val_accuracy: 0.7021 - val_loss: 0.8542
Epoch 7/50
782/782 - 5s - 7ms/step - accuracy: 0.6537 - loss: 0.9841 - val_accuracy: 0.6992 - val_loss: 0.8594
Epoch 8/50
782/782 - 5s - 7ms/step - accuracy: 0.6641 - loss: 0.9525 - val_accuracy: 0.7155 - val_loss: 0.8150
Epoch 9/50
782/782 - 11s - 14ms/step - accuracy: 0.6775 - loss: 0.9153 - val_accuracy: 0.7346 - val_loss

Test loss: 0.5570  Test accuracy: 0.8136


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
model.save("/content/drive/MyDrive/Colab Notebooks/CIFAR-models/CIFAR10_VGG11_opt.h5")

Mounted at /content/drive
